In [1]:
%matplotlib inline
import pandas as pd

#Importo Decisions.csv
decisions = pd.read_csv("../Dataset 2.0/Decisions.csv")

#Da "Decisions.csv" seleziono solo le tuple riguardanti il round desiderato (1-9)
decisions1 = decisions[decisions["Round"] == 1]

#Ho effettuato un controllo esterno per assicurarmi che tutte le tuple vengano prese in considerazione

#Estraggo "ID", "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions1 = decisions1.iloc[:, [0, 1, 3, 4]]

#Da "Decisions.csv" seleziono solo le tuple riguardanti il round successivo
decisions2 = decisions[decisions["Round"] == 2]

#Estraggo "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions2 = decisions2.iloc[:, [1, 3, 4]]

#Importo TasksQualities per le task assegnate a ciascun WA
queue = pd.read_csv("../Dataset 2.0/TasksQualities.csv", sep = ";")

queue = queue.sort_values(by = "ID")


print("end")

end


In [6]:
from sklearn.linear_model import LinearRegression
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


#Creo un DataFrame vuoto nel quale concatenerò il risultato di ciascuna iterazione che sto per eseguire
totale = pd.DataFrame()

#Creo un DataFrame vuoto nel quale concatenerò i coefficienti di regressione per ogni WA
coeffs = pd.DataFrame()


for i in range(20):
    
    #Faccio coincidere in entrambe le tabelle gli wa scelti (in due round consecutivi)
    tmp1 = decisions1[decisions1["Worker Agent ID"] == i+1]
    tmp2 = decisions2[decisions2["Worker Agent ID"] == i+1]
    tmp1 = pd.merge(tmp1, tmp2, left_on = "Session ID", right_on = "Session ID")
    tmp1 = tmp1.rename(columns = {"Worker Agent ID_x": "Worker Agent ID", "Worker Agent Reputation_x": "Worker Agent Starting Reputation", "Worker Agent Reputation_y": "Worker Agent Observed Reputation"})
    tmp1 = pd.merge(queue, tmp1, left_on = "ID", right_on = "ID")
    tmp1 = tmp1.drop("Worker Agent ID_y", 1)
    tmp1 = tmp1.drop("Session ID", 1)
    tmp1 = tmp1.sort_values(by = "ID")
    
    
    #Preparazione e training del modello per ogni wa riguardo al round scelto
    xtmp = tmp1.iloc[:, 0:93]
    ytmp = tmp1.iloc[:, 93]
    
    reg = LinearRegression().fit(xtmp, ytmp)
    #Salvataggio dei coefficienti in un Dataframe Unico
    var = pd.DataFrame(reg.coef_, columns = [f"{i + 1}"])
    
    
    totale = pd.concat([totale, tmp1])
    
    #Non usare concat ma giustapporre
    coeffs[f"{i+1}"] =  var
    coeffs.to_csv("../Coefficienti di Regressione/coeffs.csv", index = False)
    
print("end")

end


In [27]:
totale = totale.sort_values(by = "ID")

#Preparazione del dataset per il training
x = totale.iloc[:, 0:93]
y = totale.iloc[:, 93]
print("fine")

fine


In [28]:
#Training del modello utilizzando la Regressione Lineare

reg = LinearRegression().fit(x, y)
reg.score(x, y)

0.2394294578681999

In [31]:
coeffs

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,1.135297e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.467271e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2.934543e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.467271e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-6.104951e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004134
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000827
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004134
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [22]:
#Training del modello utilizzando SVR

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(x, y)
regr.score(x, y)

0.788342762977449